In [116]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
from scipy import sparse, io
import numpy as np

In [118]:
import h5sparse, h5py

In [119]:
data = io.loadmat('../download/poli_large.mat')
A = data['Problem']['A'][0][0]

In [122]:
b = A[:,1231]

In [123]:
x = sparse.linalg.gmres(A=A, b=b.toarray())[0]

In [124]:
x = sparse.csc_matrix(x).T

# Write to HDF5

In [140]:
def write_sparse_hdf5(arr, name):
    m, n = arr.shape
    nnz = arr.nnz
    with h5py.File("h5/{name}.h5".format(name=name), 'w') as f:
        arr_coo = arr.tocoo()
        
        m_dset = f.create_dataset('m', (1,), 'i')
        m_dset[...] = m
        n_dset = f.create_dataset('n', (1,), 'i')
        n_dset[...] = n
        nnz_dset = f.create_dataset('nnz', (1,), 'i')
        nnz_dset[...] = nnz
        
        row_dset = f.create_dataset('row', (nnz,), 'i')
        row_dset[...] = arr_coo.row
        col_dset = f.create_dataset('col', (nnz,), 'i')
        col_dset[...] = arr_coo.col
        data_dset = f.create_dataset('data', (nnz,), 'd')
        data_dset[...] = arr_coo.data
        

In [141]:
write_sparse_hdf5(A, 'A')
write_sparse_hdf5(b, 'b')
write_sparse_hdf5(x, 'x')

In [144]:
with h5py.File("h5/A.h5") as f_read:
    quants = ['m', 'n', 'nnz', 'row', 'col', 'data']
    for quant in quants:
        print("{} = {}".format(quant, f_read[quant].value))

m = [15575]
n = [15575]
nnz = [33033]
row = [    0     0     1 ..., 15573     0 15574]
col = [    0     1     1 ..., 15573 15574 15574]
data = [ 1.   -0.25  1.   ...,  1.   -0.02  1.  ]


In [127]:
f_read = h5py.File("A.h5")

In [128]:
d = f_read['data']

In [129]:
d.value

array([ 1.  , -0.25,  1.  , ...,  1.  , -0.02,  1.  ])

In [130]:
d.value.nonzero()

(array([    0,     1,     2, ..., 33030, 33031, 33032]),)

In [131]:
f_read.close()